# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Source data files
cities_latlng_weather_data_path = "../Resources/cities.csv"

# Read the city, et.al. data and display for study
city_weather_df = pd.read_csv(cities_latlng_weather_data_path)
city_weather_df.drop(columns=["Search Lats", "Search Lngs", "Distance (mi)", "Timestamp"], inplace=True)
city_weather_df.rename(columns={"Actual Lats": "Latitude (°)", "Actual Lngs": "Longitude (°)", "Max Temp (F)": "Max Temp (°F)"}, inplace=True)
city_weather_df.head()

,City-Country,City,Country,Latitude (°),Longitude (°),Max Temp (°F),Humidity (%),Cloud Cover (%),Wind Speed (mph),Date,Hemisphere
0,"catumbela, ao",catumbela,ao,-12.421128,13.549476,73.99,82,97,4.38,2020-11-20 05:07:25,Southern
1,"sakhon nakhon, th",sakhon nakhon,th,17.166421,104.148606,84.20,62,20,5.82,2020-11-20 05:07:26,Northern
2,"qeshm, ir",qeshm,ir,26.951627,56.272928,71.60,35,20,6.93,2020-11-20 05:07:27,Northern
3,"valdivia, cl",valdivia,cl,-39.833581,-73.215417,55.40,87,90,4.70,2020-11-20 05:04:44,Southern
4,"khorixas, na",khorixas,na,-20.372065,14.959329,68.05,32,0,2.75,2020-11-20 05:07:29,Southern


In [4]:
len(city_weather_df)

475

In [3]:
city_weather_df.dtypes

City-Country         object
City                 object
Country              object
Latitude (°)        float64
Longitude (°)       float64
Max Temp (°F)       float64
Humidity (%)          int64
Cloud Cover (%)       int64
Wind Speed (mph)    float64
Date                 object
Hemisphere           object
dtype: object

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
locations = city_weather_df[["Latitude (°)", "Longitude (°)"]]
weights = city_weather_df["Humidity (%)"]
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# NOTE: The figure above did not display as currently configured.  In fact, it did not display.
#    There was also no error message.  Data Boot Camp instructions and GTS indicated this is a 
#    common issue.  Stockoverflow provided the solution below.  The solution author recommended 
#    executing the following Command line scripts:
#
#       jupyter nbextension enable --py gmaps
#       jupyter nbextension enable --py widgetsnbextension
#
# I am colorblind, so it's kinda hard for me to see if the data is fully represented in this 
#    heatmap.  Somewhat disappointing is the fact that the screen resolution of the heatmap is 
#    not so great.  That said, when zooming in slightly, country borders come out sharp.  So, I
#    hazard that is about as good as one can hope for without a commercial license.

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions:
     * A max temperature equal to or lower than 80 degrees but equal to or higher than 70.
     * Wind speed less than 10 mph.
     * Humidity less than 60%.
     * Zero cloudiness.
* Drop any rows will null values.

In [40]:
# Calculate dataframe index values where conditions met
idx = np.where(
    (city_weather_df["Max Temp (°F)"] >= 70) &
    (city_weather_df["Max Temp (°F)"] <= 80) &
    (city_weather_df["Wind Speed (mph)"] < 10) &
    (city_weather_df["Humidity (%)"] < 60) &
    (city_weather_df["Cloud Cover (%)"] == 0))

# Filter for only rows with computed index values
ideal_locations_df = city_weather_df.loc[idx]

# Drop any rows with null values
ideal_locations_df.dropna(inplace=True)

ideal_locations_df.head()

,City-Country,City,Country,Latitude (°),Longitude (°),Max Temp (°F),Humidity (%),Cloud Cover (%),Wind Speed (mph),Date,Hemisphere
6,"lashio, mm",lashio,mm,22.966458,97.752535,76.78,47,0,1.92,2020-11-20 05:07:31,Northern
37,"napasar, in",napasar,in,27.965266,73.563194,71.47,21,0,1.92,2020-11-20 05:08:09,Northern
116,"karachi, pk",karachi,pk,24.860734,67.001136,71.60,33,0,8.05,2020-11-20 05:09:32,Northern
132,"johi, pk",johi,pk,26.694083,67.621518,72.19,21,0,4.92,2020-11-20 05:10:01,Northern
319,"bandar-e lengeh, ir",bandar-e lengeh,ir,26.562787,54.888679,75.20,44,0,6.93,2020-11-20 05:13:38,Northern


In [39]:
# Count the number of ideal locations
len(ideal_locations_df)

8

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates. (This was already done in WeatherPy and retained in the dataframe.)
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [36]:
# Fixed parameters that will be fed into the Google Places API query parameters
# search_terms = "hotel"
search_radius = 5000
search_type = "lodging"

# set up empty lists to hold reponse info
hotel_list = []
success_lat_list = []
success_lng_list = []

for index, row in ideal_locations_df.iterrows():
    
    # coordinates - Used to search for Nearest City
    lat = row["Latitude (°)"]
    lng = row["Longitude (°)"]


    # Try to extract one hotel name for each ideal weather city
    try:
        # set up a parameters dictionary
        params = {
            "location": f"{lat}, {lng}",
            "radius": search_radius,
            "type": search_type,
            "key": g_key
        }
        
        #"keyword": search_terms,
        
        # base url
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        
        # run a request using params dictionary
        response = requests.get(base_url, params=params)
        
        # convert response to json
        places_data = response.json()
        
        hotel_list.append(places_data["results"][0]["name"])
        success_lat_list.append(lat)
        success_lng_list.append(lng)
                        
        # Pause a few seconds to allow for processing delays
        # time.sleep(1) 
        
        # Printed as Check
        print(f"Hotel within 5km of geocoordinates Lat {lat}, Lng {lng} found! Appending hotel name.")

    # Handle exceptions for cities whose geocoordiantes are not returned in the Google API
    except:
        # Append null values
        print(f"Hotel near geocoordinates Lat {lat}, Lng {lng} not found")
        pass 

Hotel within 5km of geocoordinates Lat 22.966457600000002, Lng 97.75253520000001 found! Appending hotel name.
Hotel within 5km of geocoordinates Lat 27.965265999999996, Lng 73.56319409999998 found! Appending hotel name.
Hotel within 5km of geocoordinates Lat 24.8607343, Lng 67.0011364 found! Appending hotel name.
Hotel within 5km of geocoordinates Lat 26.694083000000003, Lng 67.6215179 found! Appending hotel name.
Hotel within 5km of geocoordinates Lat 26.562786699999997, Lng 54.888679 found! Appending hotel name.
Hotel near geocoordinates Lat 9.4547967, Lng 18.944280399999997 not found
Hotel within 5km of geocoordinates Lat 29.488851699999998, Lng 78.76037590000001 found! Appending hotel name.
Hotel within 5km of geocoordinates Lat 15.104832, Lng 36.6592026 found! Appending hotel name.


In [41]:
print(f"Hotels List = {len(hotel_list)}")
print(f"Successful Latitudes List = {len(success_lat_list)}")
print(f"Successful Longitudes List = {len(success_lng_list)}")

Hotels List = 7
Successful Latitudes List = 7
Successful Longitudes List = 7


In [44]:
hotel_data = {"Hotel Name": hotel_list, "Latitude (°)": success_lat_list, "Longitude (°)": success_lng_list}
hotel_df = pd.DataFrame(hotel_data)
hotel_df

,Hotel Name,Latitude (°),Longitude (°)
0,Great City Hotel,22.966458,97.752535
1,Shiv mandir,27.965266,73.563194
2,Pearl Continental Karachi,24.860734,67.001136
3,Mashuk Hotel,26.694083,67.621518
4,کافی نت پارس,26.562787,54.888679
5,Ram Medical Home,29.488852,78.760376
6,Tsegay Tilyan Hotel and Bar,15.104832,36.659203


In [45]:
# Perform inner merge on the 2 dataframes
hotel_df = pd.merge(hotel_df, ideal_locations_df, on=["Latitude (°)", "Longitude (°)"])
hotel_df.head()

,Hotel Name,Latitude (°),Longitude (°),City-Country,City,Country,Max Temp (°F),Humidity (%),Cloud Cover (%),Wind Speed (mph),Date,Hemisphere
0,Great City Hotel,22.966458,97.752535,"lashio, mm",lashio,mm,76.78,47,0,1.92,2020-11-20 05:07:31,Northern
1,Shiv mandir,27.965266,73.563194,"napasar, in",napasar,in,71.47,21,0,1.92,2020-11-20 05:08:09,Northern
2,Pearl Continental Karachi,24.860734,67.001136,"karachi, pk",karachi,pk,71.60,33,0,8.05,2020-11-20 05:09:32,Northern
3,Mashuk Hotel,26.694083,67.621518,"johi, pk",johi,pk,72.19,21,0,4.92,2020-11-20 05:10:01,Northern
4,کافی نت پارس,26.562787,54.888679,"bandar-e lengeh, ir",bandar-e lengeh,ir,75.20,44,0,6.93,2020-11-20 05:13:38,Northern


In [46]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude (°)", "Longitude (°)"]]

In [48]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))